In [1]:

import matplotlib as plt
import youtube_dl
from librosa import display

In [2]:
from src import firestore

db = firestore.init_firestore()
songs_ref = db.collection("songs")

# Get the song with id "my-id"
song_ref = songs_ref.document("spotify-53riUuAdYqkYRRMOZYVcuZ")
song_doc = song_ref.get().to_dict()

In [18]:
song_doc

{'language': 'sw',
 'lyrics': '[Vers 1]<br/>En ensam kvinna söker en man<br/>Svar till: Hjärtat talar sant<br/>En pessimist i sitt livs form<br/>Jag vädrar blod, det luktar sorg<br/><br/>[Refräng]<br/>En utomjordings kärlekstörst<br/>En undran vem som svek vem först<br/>Ät fett och socker tills du spyr<br/>Eller blir en fyratonsmartyr<br/>Sälj dig, sälj dig dyrt<br/><br/>[Vers 2]<br/><a class="ReferentFragmentdesktop__ClickTarget-sc-110r0d9-0 cesxpW" href="/13590978/Kent-dom-andra/Kom-lana-tornekronan-min"><span class="ReferentFragmentdesktop__Highlight-sc-110r0d9-1 jAzSMw">Kom låna törnekronan min</span></a><span style="position:absolute;opacity:0;width:0;height:0;pointer-events:none;z-index:-1" tabindex="0"></span><span><span style="position:absolute;opacity:0;width:0;height:0;pointer-events:none;z-index:-1" tabindex="0"></span><span style="position:absolute;opacity:0;width:0;height:0;pointer-events:none;z-index:-1" tabindex="0"></span></span><br/>Lid för konsten eller brinn<br/>Jag 

In [3]:
# downloads yt_url to the same directory from which the script runs
def download_audio(yt_url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
    }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download([yt_url])

def main():
    yt_url = "https://www.youtube.com/watch?v=8OAPLk20epo"
    download_audio(yt_url)

# main()

In [4]:
import pytube 
yt = pytube.YouTube(f"https://www.youtube.com/watch?v={song_doc['youtube_id']}")

In [5]:
yt.streams.filter(type="audio", mime_type="audio/webm").order_by("abr").desc().first().download(f"../data/songs/", filename=f"{song_doc['youtube_id']}.webm")

'/Users/robert-andreidamian/Workspace/personal/language-music/language-music-backend/notebooks/../data/songs/fWxUr_vDwwE.webm'

In [6]:
import subprocess

def convert_and_split(filename):
    output_file_name = '/'.join(filename.split('/')[:-1]) + "/" + filename.split('/')[-1].split('.')[0] + ".wav"
    command = ['ffmpeg', '-y', '-i', filename, '-ac',  '2', '-vn', output_file_name]
    subprocess.run(command,stdout=subprocess.PIPE,stdin=subprocess.PIPE)


In [7]:
convert_and_split("../data/songs/fWxUr_vDwwE.webm")

ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.1.1_2 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr 

In [1]:
input_file = "../data/songs/fWxUr_vDwwE.wav"

In [2]:
from asteroid.models import BaseModel
import soundfile as sf

# 'from_pretrained' automatically uses the right model class (asteroid.models.DPRNNTasNet).
model = BaseModel.from_pretrained("mpariente/DPRNNTasNet-ks2_WHAM_sepclean")

# You can pass a NumPy array:
mixture, _ = sf.read(input_file, dtype="float32", always_2d=True)
# Soundfile returns the mixture as shape (time, channels), and Asteroid expects (batch, channels, time)
mixture = mixture.transpose()
mixture = mixture.reshape(1, mixture.shape[0], mixture.shape[1])


In [ ]:
out_wavs = model.separate(mixture)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display


def show_magspec(waveform, **kw):
    return librosa.display.specshow(
        librosa.amplitude_to_db(np.abs(librosa.stft(waveform))),
        y_axis="log", x_axis="time",
        **kw
    )


est1 = sf.read("female-female-mixture_est1.wav")[0]
est2 = sf.read("female-female-mixture_est2.wav")[0]

fig, ax = plt.subplots(1, 2, figsize=(15, 5))
show_magspec(est1, sr=8000, ax=ax[0])
show_magspec(est2, sr=8000, ax=ax[1])

# With Spleeter

In [1]:
from scipy.io import wavfile

input_wav = wavfile.read("../data/songs/fWxUr_vDwwE.wav")

In [ ]:
from spleeter.separator import Separator

separator = Separator('spleeter:2stems')
separated_output = separator.separate(input_wav[1])

In [9]:

import torch
import torchaudio
import matplotlib.pyplot as plt
from torchaudio.pipelines import HDEMUCS_HIGH_MUSDB_PLUS

bundle = HDEMUCS_HIGH_MUSDB_PLUS

model = bundle.get_model()

device = torch.device("cpu")

model.to(device)

sample_rate = bundle.sample_rate

print(f"Sample rate: {sample_rate}")


from torchaudio.transforms import Fade


def separate_sources(
    model,
    mix,
    segment=10.0,
    overlap=0.1,
    device=None,
):
    """
    Apply model to a given mixture. Use fade, and add segments together in order to add model segment by segment.

    Args:
        segment (int): segment length in seconds
        device (torch.device, str, or None): if provided, device on which to
            execute the computation, otherwise `mix.device` is assumed.
            When `device` is different from `mix.device`, only local computations will
            be on `device`, while the entire tracks will be stored on `mix.device`.
    """
    if device is None:
        device = mix.device
    else:
        device = torch.device(device)

    batch, channels, length = mix.shape

    chunk_len = int(sample_rate * segment * (1 + overlap))
    start = 0
    end = chunk_len
    overlap_frames = overlap * sample_rate
    fade = Fade(fade_in_len=0, fade_out_len=int(overlap_frames), fade_shape="linear")

    final = torch.zeros(batch, len(model.sources), channels, length, device=device)

    while start < length - overlap_frames:
        chunk = mix[:, :, start:end]
        with torch.no_grad():
            out = model.forward(chunk)
        out = fade(out)
        final[:, :, :, start:end] += out
        if start == 0:
            fade.fade_in_len = int(overlap_frames)
            start += int(chunk_len - overlap_frames)
        else:
            start += chunk_len
        end += chunk_len
        if end >= length:
            fade.fade_out_len = 0
    return final


def plot_spectrogram(stft, title="Spectrogram"):
    magnitude = stft.abs()
    spectrogram = 20 * torch.log10(magnitude + 1e-8).numpy()
    _, axis = plt.subplots(1, 1)
    axis.imshow(spectrogram, cmap="viridis", vmin=-60, vmax=0, origin="lower", aspect="auto")
    axis.set_title(title)
    plt.tight_layout()




# We download the audio file from our storage. Feel free to download another file and use audio from a specific path
SAMPLE_SONG = "../data/songs/fWxUr_vDwwE.wav"
waveform, sample_rate = torchaudio.load(SAMPLE_SONG)  # replace SAMPLE_SONG with desired path for different song
waveform = waveform.to(device)
mixture = waveform

# parameters
segment: int = 10
overlap = 0.1

print("Separating track")

ref = waveform.mean(0)
waveform = (waveform - ref.mean()) / ref.std()  # normalization

sources = separate_sources(
    model,
    waveform[None],
    device=device,
    segment=segment,
    overlap=overlap,
)[0]
sources = sources * ref.std() + ref.mean()

sources_list = model.sources
sources = list(sources)

audios = dict(zip(sources_list, sources))



Sample rate: 44100
Separating track


In [10]:
torchaudio.save("../data/songs/fWxUr_vDwwE-vocals.wav", audios["vocals"], sample_rate)

In [1]:
import pydub
import pydub.silence

sound = pydub.AudioSegment.from_file("../data/songs/fWxUr_vDwwE/vocals.wav", format="wav")
silent_ranges = pydub.silence.detect_silence(
    sound, 
    min_silence_len=1000,
    silence_thresh=-40,
)


In [15]:
silent_ranges

[[0, 21438],
 [21758, 40079],
 [87769, 91411],
 [91657, 93237],
 [94447, 96014],
 [96321, 100525],
 [142742, 153981],
 [154035, 159314],
 [187417, 189767],
 [190242, 191487],
 [217543, 223451],
 [223494, 245233],
 [245259, 248389]]

In [2]:
chunks = pydub.silence.split_on_silence(sound, min_silence_len=5000, silence_thresh=-32, keep_silence=1000)

In [4]:
pydub.silence.detect_nonsilent(sound, min_silence_len=5000, silence_thresh=-32)

[[41686, 81610],
 [101164, 140991],
 [160557, 184367],
 [192568, 216218],
 [250873, 251809]]

In [3]:
chunks

In [5]:
from pydub.playback import play

In [16]:
play(chunks[0])

Input #0, wav, from '/var/folders/n7/47q7nnr50llfkvhdcw2r73m80000gn/T/tmp2nws7lk6.wav':
  Duration: 00:00:41.92, bitrate: 1411 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 44100 Hz, 2 channels, s16, 1411 kb/s


In [4]:
import whisper_timestamped

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.


In [11]:
import numpy as np
first_chunk_array = np.array(chunks[0].get_array_of_samples())

In [12]:
first_chunk_array

array([   4, -202,  302, ..., -731, -724, -712], dtype=int16)

In [13]:
np.savetxt("first_chunk.wav", first_chunk_array, fmt="%d", delimiter=",", newline="\n")

In [17]:
from scipy.io.wavfile import write 
write("first_chunk.wav", chunks[0].frame_rate, np.fromstring(chunks[0]))

TypeError: a bytes-like object is required, not 'AudioSegment'

In [26]:
audio = chunks[0].set_frame_rate(16000)

In [27]:
audio_array = np.array(audio.get_array_of_samples(), dtype=np.float32).reshape((-1, audio.channels)) / (
            1 << (8 * audio.sample_width - 1)), audio.frame_rate

In [28]:
write("first_chunk.wav", 16000, audio_array[0])